## Blog-Reading Chatbot with Qdrant, GPT-4o and LangChain

This notebook demonstrates how to use LangChain and Qdrant to build a RAG model that processes blog data. 
The RAG model is a retrieval-augmented model that uses a retriever to find relevant documents and a generator to generate answers. 
The retriever is built using Qdrant, an open-source vector search engine.
For the generator, we are using GPT-4o, OpenAI's most recent language model.

## Implementation

You can install the required libraries using the following command - get Poetry from [here](https://python-poetry.org/docs/):

```bash
poetry install
```

In [ ]:
import getpass
import os

import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Qdrant
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [ ]:
llm = ChatOpenAI(model="gpt-4o")

## Download and Index

You need to first load the blog post contents. To do this, use urllib and BeautifulSoup to load and parse the HTML content. Then index the blog post contents using Qdrant.

In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()


### Chunk Data Before Indexing

The loaded document is over 42k characters long. This is too long to fit in the context window of many models. Even for those models that could fit the full post in their context window, models can struggle to find information in very long inputs.

To handle this, you have to split the document into chunks for embedding and vector storage. This should help you retrieve only the most relevant bits of the blog post at run time.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Qdrant.from_documents(
    documents=splits, embedding=OpenAIEmbeddings(), location=":memory:", collection_name="lilianweng"
)

Use LangChain's LCEL Runnable protocol to define the chain: 1) context, 2) format_docs, 3) prompt and 4) output parsing. 
Then, run the chain to get the answer. 

Download a pre-written prompt from the LangChain hub and run it with the blog post content. 
The prompt is designed to extract the most relevant information from the blog post.

In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is Task Decomposition?")